In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
import aim

from library.epoch import evaluate_model
from library.model import SSLVisualClassifier, VisualClassifier, AlexNetwork
from library.dataset import SSLDataset
from library.epoch import train_ssl_epoch


os.environ["TOKENIZERS_PARALLELISM"] = "false"


# Model Configuration Dictionary
model_configs = {
    "ViT": {
        "ssl_class": SSLVisualClassifier,
        "ssl_args": {
            "vision_model_name": "google/vit-base-patch16-224-in21k",
            "num_classes": 8,
        },
        "weights_path": "ssl_pretrained_vit.pth",
        "feature_size": 768,  # ViT-base hidden size
    },
    "AlexNet": {
        "ssl_class": AlexNetwork,
        "ssl_args": {},
        "weights_path": "ssl_pretrained_alex.pth",
        "feature_size": 4096,  # fc6 output size
    },
}

/data/kazanplova/anaconda3/envs/flux_train/lib/python3.11/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
def create_subset(dataset, n_percent=100, seed=42):
    np.random.seed(seed)
    n_total = len(dataset)
    n_samples = int(n_total * n_percent / 100)
    indices = np.random.choice(n_total, n_samples, replace=False)
    return Subset(dataset, indices)


def run_experiments():
    # SSL pretraining first
    for model_name, config in model_configs.items():
        if not os.path.exists(config["weights_path"]):
            print(f"SSL Pretraining {model_name}...")
            ssl_model = config["ssl_class"](**config["ssl_args"]).to(device)
            ssl_optimizer = optim.Adam(ssl_model.parameters(), lr=1e-4)

            # Create SSL dataset
            full_train_dataset = datasets.Flowers102(
                root="./data", split="train", transform=train_transform, download=True
            )
            ssl_dataset = SSLDataset(full_train_dataset)
            ssl_loader = DataLoader(ssl_dataset, batch_size=256, shuffle=True)

            # Train SSL
            ssl_run = aim.Run(repo=".", experiment=f"{model_name}_ssl_pretrain")
            num_epochs_ssl = 25
            for epoch in range(num_epochs_ssl):
                metrics = train_ssl_epoch(
                    ssl_model,
                    ssl_loader,
                    ssl_optimizer,
                    nn.CrossEntropyLoss(),
                    device,
                    ssl_run,
                    epoch,
                )
                print(
                    f"{model_name} SSL Epoch {epoch + 1}/{num_epochs_ssl} | Loss: {metrics['loss']:.4f} | Acc: {metrics['accuracy']:.4f}"
                )
            if model_name == "ViT":
                torch.save(ssl_model.vision_model.state_dict(), config["weights_path"])
            else:
                torch.save(ssl_model.state_dict(), config["weights_path"])
            ssl_run.close()

    for model_name in model_configs.keys():
        for n_percent in [100, 50, 10]:
            for use_ssl in [True, False]:
                experiment_type = "ssl" if use_ssl else "baseline"
                experiment_name = f"{model_name}_{experiment_type}_{n_percent}pct"

                full_train_dataset = datasets.Flowers102(
                    root="./data", split="train", transform=train_transform
                )
                train_subset = create_subset(full_train_dataset, n_percent)
                train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)

                model = VisualClassifier(
                    model_name=model_name,
                    num_classes=102,
                    hidden_dim=1024,
                    model_configs=model_configs,
                    use_ssl=use_ssl,
                ).to(device)

                optimizer = optim.AdamW(model.classifier.parameters(), lr=1e-4)
                run = aim.Run(repo=".", experiment=experiment_name)
                run["hparams"] = {
                    "model": model_name,
                    "data_percent": n_percent,
                    "ssl_used": use_ssl,
                }

                best = 0
                for epoch in tqdm(
                    range(25),
                    leave=False,
                    desc=f"Training {model_name} on {n_percent} of data with ssl={use_ssl}",
                ):
                    model.train()
                    total_loss = 0
                    all_preds, all_labels = [], []

                    for images, labels in tqdm(train_loader, leave=False):
                        images, labels = images.to(device), labels.to(device)

                        optimizer.zero_grad()
                        outputs = model(images)
                        loss = nn.CrossEntropyLoss()(outputs, labels)
                        loss.backward()
                        optimizer.step()

                        total_loss += loss.item()
                        preds = torch.argmax(outputs, dim=1)
                        all_preds.append(preds.cpu())
                        all_labels.append(labels.cpu())

                    all_preds = torch.cat(all_preds).numpy()
                    all_labels = torch.cat(all_labels).numpy()
                    train_accuracy = accuracy_score(all_labels, all_preds)
                    val_metrics = evaluate_model(
                        model, val_loader, device, num_classes=num_classes
                    )
                    print(val_metrics)
                    run.track(total_loss / len(train_loader), name="loss", epoch=epoch)
                    run.track(train_accuracy, epoch=epoch, name="Train Accuracy")
                    run.track(val_metrics, epoch=epoch)

                    if val_metrics["f1_score_weighted"] > best:
                        best = val_metrics["f1_score_weighted"]
                        torch.save(model.state_dict(), f"{experiment_name}_best.pth")

                model.load_state_dict(torch.load(f"{experiment_name}_best.pth"))
                test_metrics = evaluate_model(
                    model, test_loader, device, num_classes=num_classes
                )
                print(test_metrics)
                run.track(test_metrics)
                run.close()

In [4]:
train_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

val_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

device = torch.device("cuda:0")
ssl_run = aim.Run(repo=".", experiment="flowers102_ssl_pretrain")
ssl_model = SSLVisualClassifier("google/vit-base-patch16-224-in21k").to(device)
ssl_optimizer = optim.Adam(ssl_model.parameters(), lr=1e-4)
ssl_criterion = nn.CrossEntropyLoss()

full_train_dataset = datasets.Flowers102(
    root="./data", split="train", transform=train_transform, download=True
)

val_dataset = datasets.Flowers102(
    root="./data", split="val", transform=train_transform, download=True
)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True, num_workers=0)

test_dataset = datasets.Flowers102(
    root="./data", split="test", transform=train_transform, download=True
)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=0)

num_classes = 102

In [5]:
run_experiments()

SSL Pretraining ViT...


  0%|          | 0/4 [00:00<?, ?it/s]

ViT SSL Epoch 1/25 | Loss: 2.0906 | Acc: 0.1304


  0%|          | 0/4 [00:00<?, ?it/s]

ViT SSL Epoch 2/25 | Loss: 2.0560 | Acc: 0.1696


...

  0%|          | 0/16 [00:00<?, ?it/s]

{'accuracy': 0.05196078431372549, 'precision_weighted': 0.01709490454014992, 'recall_weighted': 0.05196078431372549, 'f1_score_weighted': 0.021210539245253384, 'precision_class_0': 0.0, 'recall_class_0': 0.0, 'f1_score_class_0': 0.0, 'precision_class_1': 0.0, 'recall_class_1': 0.0, 'f1_score_class_1': 0.0, 'precision_class_2': 0.0, 'recall_class_2': 0.0, 'f1_score_class_2': 0.0, 'precision_class_3': 0.08695652173913043, 'recall_class_3': 0.2, 'f1_score_class_3': 0.12121212121212122, 'precision_class_4': 0.0, 'recall_class_4': 0.0, 'f1_score_class_4': 0.0, 'precision_class_5': 0.0, 'recall_class_5': 0.0, 'f1_score_class_5': 0.0, 'precision_class_6': 0.0, 'recall_class_6': 0.0, 'f1_score_class_6': 0.0, 'precision_class_7': 0.0, 'recall_class_7': 0.0, 'f1_score_class_7': 0.0, 'precision_class_8': 0.0, 'recall_class_8': 0.0, 'f1_score_class_8': 0.0, 'precision_class_9': 0.0, 'recall_class_9': 0.0, 'f1_score_class_9': 0.0, 'precision_class_10': 0.021739130434782608, 'recall_class_10': 0.2, 

/tmp/ipykernel_653734/2503210465.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{experiment_name}_best.pth"))


  0%|          | 0/97 [00:00<?, ?it/s]

{'accuracy': 0.04943893315986339, 'precision_weighted': 0.05594607540320196, 'recall_weighted': 0.04943893315986339, 'f1_score_weighted': 0.022019101074316174, 'precision_class_0': 0.0, 'recall_class_0': 0.0, 'f1_score_class_0': 0.0, 'precision_class_1': 0.0, 'recall_class_1': 0.0, 'f1_score_class_1': 0.0, 'precision_class_2': 0.0, 'recall_class_2': 0.0, 'f1_score_class_2': 0.0, 'precision_class_3': 0.025974025974025976, 'recall_class_3': 0.1111111111111111, 'f1_score_class_3': 0.042105263157894736, 'precision_class_4': 0.0, 'recall_class_4': 0.0, 'f1_score_class_4': 0.0, 'precision_class_5': 0.1, 'recall_class_5': 0.04, 'f1_score_class_5': 0.05714285714285714, 'precision_class_6': 0.0, 'recall_class_6': 0.0, 'f1_score_class_6': 0.0, 'precision_class_7': 0.0, 'recall_class_7': 0.0, 'f1_score_class_7': 0.0, 'precision_class_8': 0.0, 'recall_class_8': 0.0, 'f1_score_class_8': 0.0, 'precision_class_9': 0.0, 'recall_class_9': 0.0, 'f1_score_class_9': 0.0, 'precision_class_10': 0.016, 'reca